# Extracción de datos

## Importaciones

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse, parse_qs


### Sitio web
Se obtendrán datos de proyectos de ley en Colombia.

In [2]:
url = 'https://www.camara.gov.co/secretaria/proyectos-de-ley?p=secretaria/proyectos-de-ley&field_periodo_legislativo_target_id=All&field_estadoley_target_id=All&field_fastrack_value=All&field_tipo_taxonomia_target_id=All&field_origen_target_id=All&field_comision_proyecto_de_ley_target_id=All&combine=&page='
base_url = 'https://www.camara.gov.co/secretaria/proyectos-de-ley?p=secretaria/proyectos-de-ley&field_periodo_legislativo_target_id=All&field_estadoley_target_id=All&field_fastrack_value=All&field_tipo_taxonomia_target_id=All&field_origen_target_id=All&field_comision_proyecto_de_ley_target_id=All&combine=&page=0'

### Obtención de contenido

In [3]:
def get_total_pages(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Search for the last page link
        last_link = soup.find('li', class_='pager__item--last')

        if last_link:
            # Extract the URL of the last page link
            last_url = last_link.find('a')['href']

            # Search for the 'page' parameter in the URL
            query_params = parse_qs(urlparse(last_url).query)

            # Get the last page number
            last_page = int(query_params.get('page', [0])[0]) + 1  # Add 1 to get the total number of pages
            return last_page
        else:
            print("The link to the last page was not found.")
            return None
    else:
        print(f"Error obtaining page: {response.status_code}")
        return None

# Get the total number of pages
total_pages = get_total_pages(base_url)
total_pages


119

In [4]:
def scrape_page(page_number):
    page_url = url + str(page_number)
    response = requests.get(page_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        table = soup.find('table', class_='table cols-9')
        if table:
            header = [th.text.strip() for th in table.find_all('th')]
            rows = table.find_all('tr')[1:]  # Ignore header

            # Extract data from rows
            data = []
            for row in rows:
                cells = row.find_all('td')
                if cells:
                    row_data = [cell.text.strip() for cell in cells]
                    data.append(row_data)

            return header, data
        else:
            print(f"Table not found on page {page_number}.")
            return None, None
    else:
        print(f"Error obtaining page {page_number}: {response.status_code}")
        return None, None
    
all_data = []

for page in range(total_pages):
    print(f"scraping page {page+1}")
    header, data = scrape_page(page)
    if header and data:
        all_data.extend(data)

# Create a DataFrame
if all_data:
    df = pd.DataFrame(all_data, columns=header)
    df
else:
    print("No data found.")

scraping page 1
scraping page 2
scraping page 3
scraping page 4
scraping page 5
scraping page 6
scraping page 7
scraping page 8
scraping page 9
scraping page 10
scraping page 11
scraping page 12
scraping page 13
scraping page 14
scraping page 15
scraping page 16
scraping page 17
scraping page 18
scraping page 19
scraping page 20
scraping page 21
scraping page 22
scraping page 23
scraping page 24
scraping page 25
scraping page 26
scraping page 27
scraping page 28
scraping page 29
scraping page 30
scraping page 31
scraping page 32
scraping page 33
scraping page 34
scraping page 35
scraping page 36
scraping page 37
scraping page 38
scraping page 39
scraping page 40
scraping page 41
scraping page 42
scraping page 43
scraping page 44
scraping page 45
scraping page 46
scraping page 47
scraping page 48
scraping page 49
scraping page 50
scraping page 51
scraping page 52
scraping page 53
scraping page 54
scraping page 55
scraping page 56
scraping page 57
scraping page 58
scraping page 59
scrapi

### Análisis de contenido

In [13]:
df.head()


,No. Cámara,No. Senado,Proyecto,Tipo,Autor,Estado,Comisión,Origen,Legislatura
0,228/2025C,,CONCURSO DEPARTAMENTAL BANDAS SAMANIEGO,Ley Ordinaria,"Ermes Evelio Pete Vivas, Etna Tamara Argote Ca...",Trámite en Comisión,,Cámara,2025 - 2026
1,227/2025C,,REFORMA AL DEPORTE,Ley Ordinaria,"Juan Camilo Londoño Barrera, Aníbal Gustavo Ho...",Trámite en Comisión,,Cámara,2025 - 2026
2,226/2025C,017/2024S,PATRIMONIO CULTURAL SANTA CRUZ DE MOMPOX,Ley Ordinaria,Enrique Cabrales Baquero,Trámite en Comisión,Comisión Sexta o de Transportes y Comunicaciones,Senado,2025 - 2026
3,225/2025C,,LICENCIA DE MATERNIDAD LN Y R,Ley Ordinaria,Aníbal Gustavo Hoyos Franco,Trámite en Comisión,,Cámara,2025 - 2026
4,224/2025C,361/2024S,250 AÑOS DE EL CARMEN DE BOLIVAR,Ley Ordinaria,Enrique Cabrales Baquero,Trámite en Comisión,Comisión Segunda o de Relaciones Internacionales,Senado,2025 - 2026


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5930 entries, 0 to 5929
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   No. Cámara   5930 non-null   object
 1   No. Senado   5930 non-null   object
 2   Proyecto     5930 non-null   object
 3   Tipo         5930 non-null   object
 4   Autor        5930 non-null   object
 5   Estado       5930 non-null   object
 6   Comisión     5930 non-null   object
 7   Origen       5930 non-null   object
 8   Legislatura  5930 non-null   object
dtypes: object(9)
memory usage: 417.1+ KB


In [16]:
df.shape

(5930, 9)

In [19]:
df.to_csv('./data/proyectos_de_ley_colombia.csv', index=False, encoding='utf-8-sig')